# Let AI Search Web for Answers

Give your AI the ability to search the web and use tools.


In [ ]:
%pip install -qU pixeltable boto3 duckduckgo-search


In [ ]:
# Set up AWS credentials for Bedrock
import os, getpass
if 'AWS_ACCESS_KEY_ID' not in os.environ:
    os.environ['AWS_ACCESS_KEY_ID'] = getpass.getpass('AWS Access Key:')
    os.environ['AWS_SECRET_ACCESS_KEY'] = getpass.getpass('AWS Secret Key:')
    os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'


In [ ]:
import pixeltable as pxt
from pixeltable.functions.bedrock import converse, invoke_tools
from duckduckgo_search import DDGS


In [ ]:
# Step 1: Define web search tool
@pxt.udf
def search_web(query: str, max_results: int = 3) -> str:
    """Search the web and return results."""
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))
    return '\n'.join(f"{r['title']}: {r['body']}" for r in results)


In [ ]:
# Step 2: Register tools and create agent table
tools = pxt.tools(search_web)
pxt.create_dir('agent', if_exists='ignore')
agent = pxt.create_table('agent.qa', {'question': pxt.String}, if_exists='ignore')


In [ ]:
# Step 3: Add AI tool calling
agent.add_computed_column(if_exists='ignore',
    initial_response=converse(
        model_id='amazon.nova-pro-v1:0',
        messages=[{'role': 'user', 'content': [{'text': agent.question}]}],
        tool_config=tools
    ))
agent.add_computed_column(tool_output=invoke_tools(tools, agent.initial_response), if_exists='ignore')


In [ ]:
# Step 4: Generate final answer
agent.add_computed_column(if_exists='ignore',
    final_response=converse(
        model_id='amazon.nova-pro-v1:0',
        messages=[{'role': 'user', 'content': [{'text': 
            f"Question: {agent.question}\nTool Results: {agent.tool_output}"}]}]
    ))
agent.add_computed_column(answer=agent.final_response.output.message.content[0].text, if_exists='ignore')


In [ ]:
# Ask questions - AI will search web automatically
agent.insert([{'question': "What's the latest news about SpaceX?"}])


In [ ]:
# View results
agent.select(agent.question, agent.answer).head()


**What's Happening:**
- Define Python function as tool with `@pxt.udf`
- Register tools with `pxt.tools()`
- AI decides when to call tools
- `invoke_tools()` executes the selected tool
- Second LLM call synthesizes final answer

**Variation:** Add more tools (weather, stock data, calculator):
```python
tools = pxt.tools(search_web, get_weather, fetch_stock_price)
```

**Next:** `answer-questions-from-docs.ipynb` • `build-chatbot-with-memory.ipynb`
